# Script to prompt LLMs

In [1]:
# import packages
import requests
import json
import pandas as pd
import os
import openai
from openai import OpenAI
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

In [2]:
# If OpenAI used

# Get API key from the environment variable
api_key = os.environ.get('OPENAI_API_KEY')

# Set the API key
openai.api_key = api_key

In [13]:
# Define function to create a prompt
def create_prompt(object_name):
    #prompt = f"Generate alternative uses for the object [{object_name}]."
    prompt = f"""
        You are meant to assist students in group ideation. They are asked to propose alternative
        uses for an object and you should propose yours to give them ideas as well as inspire them to 
        explore other uses. 
        You are a very creative, open-minded person and can propose creative, out-of-the-box ideas while staying realistic. 
        Your ideas will be even more appreciated if they are original or useful in real-life or both.
        
        Generate alternative uses for the object [{object_name}]. 
        
        Provide five alternative uses, each explained in a concise sentence and following this example: "Sock: You could put it on your hands and pretend to have superpowers".
        """
        
    return prompt


# Define function to call the LLM API
def call_openai_api(prompt):
    client = OpenAI()
    response = client.chat.completions.create(
        model="gpt-3.5-turbo-0125",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
            ]#,
        #seed=seed,
        #max_tokens=200,
        #temperature=temperature,
        )
    return response


# Define function to store the result in a JSON file
def store_result_json(object_name, alternative_uses):
    result = [{"object_name": object_name, "alternative_uses": use} for use in  alternative_uses.split("\n") if use.strip()]
    with open("alternative_uses.json", "w") as json_file:
        json.dump(result, json_file)


# Define function to create a CSV file and Pandas DataFrame
def create_csv_and_dataframe(object_name):
    with open("alternative_uses.json", "r") as json_file:
        data = json.load(json_file)
    df = pd.DataFrame(data)
    df.to_csv("alternative_uses.csv", index=False)
    
    # remove prefix
    df['alternative_uses'] = df['alternative_uses'].str.split(f'{object_name}: ').str[1]
    
    return df

# Define function to estimate the price of an OpenAI API request
def estimate_price(prompt_tokens, response_tokens):
    # Define your pricing model (e.g., price per token)
    price_per_input_token = 	0.50 / 1e6
    price_per_output_token = 1.5 / 1e6
    
    # Estimate the price based on the total number of tokens
    price = (prompt_tokens * price_per_input_token) + (response_tokens * price_per_output_token)
    return price

### Main function

In [5]:
object_name = "Chair"  # Example object for the Alternative Uses Test
prompt = create_prompt(object_name)

print(prompt)


        You are meant to assist students in group ideation. They are asked to propose alternative
        uses for an object and you should propose yours to give them ideas as well as inspire them to 
        explore other uses. 
        You are a very creative, open-minded person and can propose creative, out-of-the-box ideas while staying realistic. 
        Your ideas will be even more appreciated if they are original or useful in real-life or both.
        
        Generate alternative uses for the object [Chair]. 
        
        Provide five alternative uses, each explained in a concise sentence and following this example: "Sock: You could put it on your hands and pretend to have superpowers".
        


In [6]:
response = call_openai_api(prompt)
response_content = response.choices[0].message.content

print(response_content)

1. Chair: Transform it into a makeshift easel by draping a large canvas or a whiteboard across the backrest, creating a versatile standing surface for art or notes.

2. Chair: Repurpose it as a unique pot rack by hanging various pots and pans from the chair's arms and back, adding a touch of whimsy to your kitchen decor while saving space.

3. Chair: Use it as a clever plant display by placing different potted plants on the seat and hanging flowering plants from the back, creating a charming indoor garden feature.

4. Chair: Convert it into a functional bedside table by positioning a lamp and your nighttime essentials on the seat and stacking books or magazines on the backrest for easy access while relaxing in bed.

5. Chair: Reinvent it as a pet's paradise by placing a cozy blanket in the seat, turning it into a comfortable nap spot for your furry friend, with the back serving as a playful climbing structure.


In [15]:
system_fingerprint = response.system_fingerprint
prompt_tokens = response.usage.prompt_tokens
completion_tokens = response.usage.total_tokens - response.usage.prompt_tokens

print(f"System fingerprint: {system_fingerprint}")
print(f"Prompt tokens: {prompt_tokens}")
print(f"Completion tokens: {completion_tokens}")

price = estimate_price(prompt_tokens, completion_tokens)
print(f"Estimated price: ${price}")

System fingerprint: fp_4f2ebda25a
Prompt tokens: 156
Completion tokens: 193
Estimated price: $0.0003675


In [17]:
store_result_json(object_name, response_content)
df = create_csv_and_dataframe(object_name)
df.head()

,object_name,alternative_uses
0,Chair,Transform it into a makeshift easel by draping...
1,Chair,Repurpose it as a unique pot rack by hanging v...
2,Chair,Use it as a clever plant display by placing di...
3,Chair,Convert it into a functional bedside table by ...
4,Chair,Reinvent it as a pet's paradise by placing a c...


## Evaluate the creativity

In [ ]:
#todo
# see if we can load the model of ocsai
